# 卷积神经网络(CNN)

卷积神经网络(CNN), 一种深度神经网络, 主要包括**卷积**与**池化操作**, 可以完成如:

- **图片分类**: 分辨一张图片是不是猫;
- **图片识别**: 识别出图片中的物体;
- **目标检测**: 识别出图片中的物体,并标注它们的位置;
- **图片风格转移**: 将图片的风格改变;

处理图片最大的问题在于, 图片像素累加起来的维度非常高, 假如是$1000\times1000$的图片, 其输入特征维度高达$1000\times1000\times3=3 billion$.

## 背景: 边缘检测

检测一个完整的物体可能经过包括: 边缘检测, 部分区域检测, 完整物体检测等几个步骤. 其中边缘检测是检测的第一步, 它会用到**卷积操作**, 卷积操作会使用一个称为**过滤器(卷积核)** 的矩阵 (下图filter), 卷积操作如下图所示:

![cnn1](./resources/cnn1.png)

**为什么卷积操作可以检测到边缘**: 可以这样想, 假设处于该图像像素分布均匀的区域,经过卷积核的卷积,-1区的乘积与1区的乘积都抵消了,这样这些区域卷积后的值都等与0. 只有那些左边右边的像素值有差异的区域可以得到非零的卷积值, 即需要检测到的边缘.

常用的过滤器还有如:**Sobel过滤器**, **Scharr过滤器**等,但是经过验证,如果真的想要检测出复杂的图像边缘, 可以把过滤器设置为可训练的参数, 最后通过反向传播来训练.

## Padding填充

按照上面的卷积操作, 如果我们有一个$n\times n$的图片, 用$f\times\ f$的过滤器卷积,那么输出维度为:

> $(n-f+1)\times(n-f+1)$

这样出现了两个缺点: (1). **每次做卷积,图像都要缩小**; (2). **过滤器到达图像边缘的次数相比于图像中心区域更少, 这意味着丢失了很多图像边缘信息**;

通过在图像周围填充一圈或几圈像素可以解决上面的问题,这种方法称为Padding. 其填充过程可由下图所示:

![cnn2](./resources/cnn2.png)

填充后, 假设在周围都填充了一个像素点, 置$p=1$, 则卷积后的尺寸计算方式如下:

> $(n+2p-f+1)\times(n+2p-f+1)$

填充方式一般有两种: 

- **Valid卷积**: 即不填充;
- **Same卷积**: 要填充, 且填充后经过卷积运算,图片的尺寸不发生变化;

## 参考文献

- [第四门课 卷积神经网络（Convolutional Neural Networks）](http://www.ai-start.com/dl2017/html/lesson4-week1.html)